In [1]:
import os
import pickle as pkl
import numpy as np
from importlib import import_module
import torch
import torch.nn as nn
import torch.nn.functional as F

In [21]:
model_path = "./data/saved_dict/TextCNN.ckpt"
vocab_path = "./data/data/vocab.pkl"
dataset = 'data'  # 数据集
embedding = 'embedding_Weibo.npz'
model_name = "TextCNN"
PAD = "<PAD>"
UNK = '<UNK>'

In [30]:
def load_vocab(vocab_path):
    if os.path.exists(vocab_path):
        vocab = pkl.load(open(vocab_path, 'rb'))
        return vocab
def stoi(vocab, s, pad_size=32):
    token = [vocab.get(x, vocab.get(UNK)) for x in s]
    seq_len = len(token)
    if len(token) < pad_size:
        token.extend([vocab.get(PAD)] * (pad_size - len(token)))
    else:
        token = token[:pad_size]
        seq_len = len(token)
    return token, [seq_len]

In [4]:
vocab = load_vocab(vocab_path)

In [5]:
x = import_module('models.' + model_name)
config = x.Config(dataset, embedding)

model = x.Model(config)
model.load_state_dict(torch.load(model_path))
model = model.to(config.device)
model.eval()

Model(
  (embedding): Embedding(4075, 300)
  (convs): ModuleList(
    (0): Conv2d(1, 256, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 256, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 256, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

In [37]:
def predic(model, x):
    with torch.no_grad():
        output = model(x)
    return output

In [53]:
s = "这个车很好啊"

In [54]:
seq, seq_len = stoi(vocab, s, pad_size=config.pad_size)

In [55]:
seq = torch.tensor([seq]).to(config.device)

In [56]:
seq

tensor([[  17,   14,  130,    7,   10,   75, 4074, 4074, 4074, 4074, 4074, 4074,
         4074, 4074, 4074, 4074, 4074, 4074, 4074, 4074, 4074, 4074, 4074, 4074,
         4074, 4074, 4074, 4074, 4074, 4074, 4074, 4074]], device='cuda:0')

In [57]:
seq_len = torch.tensor([seq_len]).to(config.device)
seq_len

tensor([[6]], device='cuda:0')

In [58]:
output = predic(model, (seq, seq_len))

In [59]:
output

tensor([[-1.5263,  1.6253]], device='cuda:0')

In [60]:
y = torch.max(output.data, 1)[1].cpu().numpy()

In [61]:
y[0]

1